# Reinforcement Learning Using OpenAI Gym's Taxi 🚕

In this introductory notebook, i'll be applying reinforcement learning (RL) to train an agent to solve the ['Taxi' environment from OpenAI Gym](https://gym.openai.com/envs/Taxi-v3/).

We'll cover:

-   A basic introduction to RL
-   Setting up OpenAI Gym & Taxi
-   Using the Q-learning algorithm to train our Taxi agent

# Before we start, what's 'Taxi'?

Taxi is one of many environments available on OpenAI Gym. These environments are used to develop and benchmark reinforcement learning algorithms.

The goal of Taxi is to pick-up passengers and drop them off at the destination in the least amount of moves. In this tutorial, you'll start with a taxi agent that takes actions randomly:

![random agent](https://drive.google.com/uc?id=1l0XizDh9eGP3gVNCjJHrC0M3DeCWI8Fj)

…and successfully apply reinforcement learning to solve the environment:

![trained agent](https://drive.google.com/uc?id=1a-OeLhXi3W-kvQuhGRyJ1dOSw4vrIBxr)

# Installing OpenAI Gym and Taxi

To install gym (and numpy for later), run the cell below:


In [ ]:
!pip install gym
!pip install numpy

Next, import gym (and additional libraries that will be useful for later):

In [ ]:
import gym
import numpy as np
import random

# used to help with visualizing in Colab
from IPython.display import display, clear_output
from time import sleep

Gym contains a large library of different environments. Let's create the Taxi-v3 environment:

In [ ]:
# create Taxi environment
env = gym.make('Taxi-v3')

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [ ]:
# create a new instance of taxi, and get the initial state
state = env.reset()

print(f"Initial state: {state}")

Next, we'll run a for-loop to cycle through the game. At each iteration, our agent will:

1. Make a random action from the action space (0 - south, 1 - north, 2 - east, 3 - west, 4 - pick-up, 5 - drop-off)
2. Receive the new state

Here's our random agent:

In [ ]:
num_steps = 99
for s in range(num_steps+1):

    clear_output(wait=True)

    print(f"step: {s} out of {num_steps}")

    # sample a random action from the list of available actions
    action = env.action_space.sample()

    # perform this action on the environment
    env.step(action)

    # print the new state
    env.render()

    sleep(0.2)

# end this instance of the taxi environment
env.close()

step: 99 out of 99


In [ ]:
state_size = env.observation_space.n  # total number of states (S)
action_size = env.action_space.n      # total number of actions (A)

# initialize a qtable with 0's for all Q-values
qtable = np.zeros((state_size, action_size))

print(f"Q table: {qtable}")

Q table: [[0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 ...
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0.]]


In [ ]:
# hyperparameters to tune
learning_rate = 0.9
discount_rate = 0.8

# dummy variables
reward = 10 # R_(t+1)
state = env.observation_space.sample()      # S_t
action = env.action_space.sample()          # A_t
new_state = env.observation_space.sample()  # S_(t+1)

# Qlearning algorithm: Q(s,a) := Q(s,a) + learning_rate * (reward + discount_rate * max Q(s',a') - Q(s,a))
qtable[state, action] += learning_rate * (reward + discount_rate * np.max(qtable[new_state,:]) - qtable[state,action])

print(f"Q-value for (state, action) pair ({state}, {action}): {qtable[state,action]}")

Q-value for (state, action) pair (161, 4): 9.0


In [ ]:
# dummy variables
episode = random.randint(0,500)
qtable = np.random.randn(env.observation_space.sample(), env.action_space.sample())

# hyperparameters
epsilon = 1.0     # probability that our agent will explore
decay_rate = 0.01 # of epsilon

if random.uniform(0,1) < epsilon:
    # explore
    action = env.action_space.sample()
else:
    # exploit
    action = np.argmax(qtable[state,:])

# epsilon decreases exponentially --> our agent will explore less and less
epsilon = np.exp(-decay_rate*episode)

# Bringing it all together

We're done with all the building blocks needed for our reinforcement learning agent. The process for training our agent will look like:

1. Initialising our Q-table with 0's for all Q-values
2. Let our agent play Taxi over a large number of games
3. Continuously update the Q-table using the Q-learning algorithm and an exploration-exploitation strategy

Here's the full implementation:

In [ ]:
class bcolors:
    RED= '\u001b[31m'
    GREEN= '\u001b[32m'
    RESET= '\u001b[0m'

# create Taxi environment
env = gym.make('Taxi-v3')

# initialize q-table
state_size = env.observation_space.n
action_size = env.action_space.n
qtable = np.zeros((state_size, action_size))

# hyperparameters
learning_rate = 0.9
discount_rate = 0.8
epsilon = 1.0
decay_rate= 0.005

# training variables
num_episodes = 2000
max_steps = 99 # per episode

print("AGENT IS TRAINING...")

for episode in range(num_episodes):

	# Reset the environment
	state = env.reset()
	step = 0
	done = False

	for step in range(max_steps):

		# Exploration-exploitation tradeoff
		if random.uniform(0,1) < epsilon:
			# Explore
			action = env.action_space.sample()
		else:
			# Exploit
			action = np.argmax(qtable[state,:])

		# Take an action and observe the reward
		new_state, reward, done, info = env.step(action)

		# Q-learning algorithm
		qtable[state,action] = qtable[state,action] + learning_rate * (reward + discount_rate * np.max(qtable[new_state,:])-qtable[state,action])

		# Update to our new state
		state = new_state

		# if done, finish episode
		if done == True:
			break

	# Decrease epsilon
	epsilon = np.exp(-decay_rate*episode)

# Get ready to watch our trained agent
clear_output()
print(f"Our Q-table: {qtable}")
print(f"Training completed over {num_episodes} episodes")
input("Press Enter to see our trained taxi agent")
sleep(1)
clear_output()

episodes_to_preview = 3
for episode in range(episodes_to_preview):

	# Reset the environment
	state = env.reset()
	step = 0
	done = False
	episode_rewards = 0

	for step in range(num_steps):
		# clear screen
		clear_output(wait=True)

		print(f"TRAINED AGENT")
		print(f"+++++EPISODE {episode+1}+++++")
		print(f"Step {step+1}")

		# Exploit
		action = np.argmax(qtable[state,:])

		# Take an action and observe the reward
		new_state, reward, done, info = env.step(action)

		# Accumulate our rewards
		episode_rewards += reward

		env.render()
		print("")
		if episode_rewards < 0:
			print(f"Score: {bcolors.RED}{episode_rewards}{bcolors.RESET}")
		else:
			print(f"Score: {bcolors.GREEN}{episode_rewards}{bcolors.RESET}")
		sleep(0.5)

		# Update to our new state
		state = new_state

		# if done, finish episode
		if done == True:
			break

# Close the Taxi environment
env.close()

TRAINED AGENT
+++++EPISODE 3+++++
Step 16

Score: 5
